In [1]:
# ChatGPT
# a sequence of tokens, how they are sequenced, we give start of sequence the model continues
# Generatively Pre-trained Transformer
# Character level language model
# Tiny shakesphere - concatenated work of all shakesphere in text filter
# We model how charcters follow if given context
# Output is a shakesphere like text
# NanoGPT . train GPT with any text 124 million params - 38 hours training time in a single 8xA100 40GB Node
#  8xA100 40GB - One machine with 8 NVIDIA A100 GPUs, each with 40 GB of VRAM, connected via NVLink or PCIe for multi-GPU training
# using OpenWebText data 
# OpenwebText has 38GB of text data (40GB using SI units) from 8,013,769 documents
# train.py is 300 line training loop
# model.py a 300 line GPT model definition
# A multilayer perceptron is a fully connected feedforward net with at least three layers (input, hidden and output)


In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [3]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4# higher rate to converge faster for such a small model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 384 # every head is 64 dimensions 
n_head = 6
n_layer = 6 # on CPU reduce number of layers and embedding dimensions. karpathy needed 15 mins in A100 GPU in 5000 iters 1.48 is his loss
dropout = 0.2 # randomly drop 20 % of the neurons
torch.manual_seed(1337)
# scaling up 1.4873 

In [5]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(f"length of dataset in characters:  {len(text)}, type of dataset: {type(text)}")

length of dataset in characters:  1115394, type of dataset: <class 'str'>


In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
set_of_text = set(text) # {'e', 'X', 'y', 'N', 'J', '3', 'o', 'w', 'b', 'U', '
list_of_text = list(set_of_text) # ['e', 'X', 'y', 'N', 'J', '3', 'o', 'w', 'b', 'U', 
print(chars) # ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size) # entire set of characters - all possible characters the model can see

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# tokenize - convert raw text to sequence of numbers based on vocabulary. this is char level tokenization. all chars
# openi - bite pair encoding . 50000 tokens- hii there - integers between 0 to 50000. sub word encoding. short seq int with large vocab or large seq integer with small vocab
# create a mapping from characters to integers
# enumerate(iterable, start=0)
# Using the enumerate() function is useful from the standpoint of its memory and computing efficiency compared to using for-loops since it returns the index and the corresponding item at one go.
# this function assigns a count incrementing by 1 to each item of an iterable and helps us track iterations while looping through that object
stoi = { ch:i for i,ch in enumerate(chars) } #  enumerate(chars). Lookup table Each char gets a number {'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4
itos = { i:ch for i,ch in enumerate(chars) } #. Reverse lookup table of itos {0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'" .. Each number gets a character
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers. For each char, outputs the corresponding number
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string. For each number outputs the correponding character and then combined as a string
print(stoi)
print(itos)
print(encode("hii there")) # list of integers that represent string
print(decode(encode("hii there"))) # reverse mapping, decode integers



{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [8]:
# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long) # text is that massive string, now it outputs a tensor a 1-1 encoding of the characters in text to numbers
print(data.shape, data.dtype) # shape is 1 x 1115394 # torch.Size([1115394]) torch.int64
print(data[:1000])
# train validation split
n = int(0.9*len(data))
train_data = data[:n]
val_data=data[n:]
# we dont feed all at once, we sample random chunks and train on chunks at once
# max chunk is blocksize

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# # we dont feed all at once, we sample random chunks and train on chunks at once
# # max chunk is blocksize 
# block_size = 8
# train_data[:block_size+1] # tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]). context of 18, 47 56 comes next, 18,47,56 context 57 comes next

In [10]:
x = train_data[:block_size] # 8 characters
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1] # for 0 to t characters, target is the corresponding t
    target = y[t]
    print(f"when input is {context} the target: {target}")

# we could predict from 1 to block size, beyond this we have to truncate
# mini batches multiple chunks stacked on top of each other

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [11]:
# Data loading
# torch.manual_seed(1337) # seed is set so it is reproducable
# batch_size = 4 # how many independent sequences will we process in parallel?
# block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # generate batch size number of random offsets# gets 4 random indices between 0 to len(data) - block_size as we need mini batches too
    x = torch.stack([data[i:i+block_size] for i in ix]) # gets 8 chars from indices. stack as rows  4x8 tensor
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])  # gets 8 chars  from index+1
    x, y = x.to(device), y.to(device) # moved to device on creation 
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

# for b in range(batch_size): # batch dimension # for each mini batch 
#     for t in range(block_size): # time dimension
#         context = xb[b, :t+1] # does the context step 0 - 0: 0,1- 1:0,1,2-2
#         target = yb[b,t]
#         print(f"when input is {context.tolist()} the target: {target}")
# #
# target = yb[b, t] → the token the model should predict at this position

# Because yb is shifted by 1 from xb, the target is always the next token after the current context.

inputs:
torch.Size([32, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54],
        [57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46],
        [43,  1, 51, 39, 63,  1, 40, 43],
        [58, 46, 43,  1, 43, 39, 56, 57],
        [39, 58, 47, 53, 52, 12,  1, 37],
        [53, 56, 43,  1, 21,  1, 41, 39],
        [50, 39, 52, 63,  1, 47, 58, 57],
        [56, 53, 63,  1, 42, 47, 42,  1],
        [39, 51,  1, 39, 44, 56, 39, 47],
        [17, 24, 21, 38, 13, 14, 17, 32],
        [ 1, 39, 52, 42,  1, 45, 43, 50],
        [ 1, 58, 46, 39, 58,  1, 42, 53],
        [ 1, 61, 53, 59, 50, 42,  1, 21],
        [59, 57, 40, 39, 52, 42,  1, 40],
        [52, 42,  8,  0,  0, 23, 21, 26],
        [45, 53, 42, 57,  0, 23, 43, 43],
        [52,  1, 61, 39, 57,  1, 51, 53],
      

In [ ]:
# Every 500 iterations (eval_interval = 500), stop training and run estimate_loss()
@torch.no_grad()  # loss calculation dont need to influence gradients # don't track gradients (saves memory and compute)
def estimate_loss():
    out = {}
    model.eval()  # turn off dropout, layernorm stats freezing, etc.
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters) 
        for k in range(eval_iters):   # Run a forward pass through the model 200 times (each with a new batch from get_batch(split))
            X, Y = get_batch(split) # get a mini-batch from train or val
            logits, loss = model(X, Y) # forward pass
            losses[k] = loss.item() # record scalar loss
        out[split] = losses.mean() # average across eval_iters
    model.train() # switch back to training mode
    return out

In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # tril variable

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei) # randomly prevent some nodes from communicating - training in ensemble of subnetworks this prevents overfit
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out


In [ ]:
class MultiHeadAttention(nn.Module):
    ''' multiple heads of attention in parallel'''
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        out = torch.cat([h(x) for h in self.heads], dim = -1) # run the heads independently and concatenate them  
        out = self.dropout(self.proj(out)) # projection is the outcome of the linear transformation of the result of concatenation
        # # Projection allows the model to combine the information from all heads in a learnable way, instead of keeping them siloed
        # got it basically, this gives the best way to combine information accumulated from the attention heads. instead of 6 times 64 we get the best factored and weighted 384
        # Without that projection, the heads’ outputs would just sit side by side, which is less expressive because the model can’t mix them
        # It’s like having six experts each give their opinion and then using a smart weighted summary to make a final decision.
        # okay after projection i have a 64 x 256 x 384 where 64 is batch, 256 is block size and 384 as sequence , what happens in dropout. does some of the values in 384 become zero randomly . i have 20 % dropout
        # After the projection, your tensor has shape (B, T, C) → (64, 256, 384). Applying dropout with 20% does the following:
        # For each element in that 384-dimensional vector at each (batch, time) position, there’s a 20% chance it will be set to zero.
        # The remaining values are scaled up by 1/(1 - 0.2) = 1.25 during training to keep the expected sum consistent.
        return   out   

In [ ]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """ # done individually on tokens

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential( # simple linear followed by non linearity
            nn.Linear(n_embd, 4 * n_embd), # the factor of 4 is as per Attention is all you need paper
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd), # this is the projection layer going back
            nn.Dropout(dropout),    # drop out neurons
        )

    def forward(self, x):
        return self.net(x)

# layer norm - batch norm  - across batch dim . zero mean 1 std dev for every columns
# layer norm - normalize rows. a 1d vector is normalized. we dont need buffers like batch norm as we are not going across layers 

In [ ]:
# Transformer has multiple blocks of multihead attention folowed by feedforward in sequence
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size) # communication
        self.ffwd = FeedFoward(n_embd) # computation
        # x′=σx−μ​⋅γ+β where 
        # μ: mean of the token’s embedding (over its 384 dims)
        # σ: standard deviation of that embedding.
        # γ, β: learned parameters (so the model can scale/shift after normalizing).
        # Keeps activations centered (mean 0) and scaled (variance 1) → avoids exploding/vanishing value
        # For each batch b and time step t, it looks at the vector x[b, t, :] of size 384
        # Computes mean and variance over those 384 values
        # Normalizes the 384 values using that mean and variance
        # This is done independently for all b and t, so 64 × 256 vectors are normalized separately
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
 

    def forward(self, x):
        x = x + self.sa(self.ln1(x))  #  # Residual connnections , transform data, skip connection. residual pathway, . gradients hop directly in backprop, residual pathway initially impact less
        # over time blocks kick in. # per token transformation
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
# Bigram model

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # a lookup table 65 number of embeddings, and 65 is embedding dim
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # identity of token a lookup table 65 number of embeddings, and 65 is embedding dim
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # position embedding, for each position in block size a embedding, 0 to block_size-1
        # self.sa_head = Head(n_embd) # self attention module
        # self.sa_heads = MultiHeadAttention(4, n_embd // 4) # i.e 4 heads of 8-dimensional self-attention. lowers loss from 2.4 to 2.28
        # self.ffwd = FeedForward(n_embed)
        # self.blocks = nn.Sequential(
        #     Block(n_embd, n_head=4),
        #     Block(n_embd, n_head=4),
        #     Block(n_embd, n_head=4),

        # )
        # set up number of Blocks based on hyperparameter n_layer 
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size) # takes n_embed as input and vocab_size as output , a simple Fully Connected Neural Network
        # print("before", self.token_embedding_table)
        '''we have 65 possible tokens (chars in dataset)
        Each token is mapped to a vector of size 65.
        E = [
            [e11, e12, ..., e165],   # embedding vector for token 0
            [e21, e22, ..., e265],   # embedding vector for token 1
            ...
            [e651, ..., e6565]       # embedding vector for token 64
            ]

        '''
    def forward(self, idx, targets=None):
        #idx and targets are both (B,T) tensor of integers
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # B ,T , C embedding for each batch x time x c
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # arange gives 0 to T-1 # T x C
        x = tok_emb + pos_emb # B, T, C
        # one block of multihead attention + ffnn
        # x = self.sa_heads(x) # apply multi head attention 
        # x = self.ffwd(x) # B, T, C
        # blocks 
        x = self.blocks(x) # B, T, C
        logits = self.lm_head(x) # B x T x vocab _size obtaining the logits from token embedding - we may have 124 or even 256 as embedding space - we use FFNN to get back
        # networks get too deep. how to optimize them. 
    
        # a vocab size logits 
        # the vocab size is 65- all characters of shakesphere. we create a random embedding of 65 x 65 . 
        # where 65 is vocab size and 65 is the embedding space. They are learnable weights
        # During training, they get adjusted so that tokens that appear in similar contexts have similar embeddings.
        # The order of tokens will matter downstream when predicting the next token, because the model sees xb[b, :t] as “context”.
        # But the embedding of each token by itself does not encode position — it’s just a vector representing that token.
        # print(logits)
        '''
        We have the input as 4 x 8 of the forward encode - and it gives the embedding for each
        ''' 
        if targets is None:
            loss = None
        else:
            # picks out the row from table B,T,C 4 x 8 x 65
            B, T, C = logits.shape
            logits = logits.view(B*T, C ) # we stack the tokens neatly one below another without messing up the order. so we have 32 rows each of 65 numbers as per embedding
            targets = targets.view (B*T) # 4 x 8 gives (32,)
            loss = F.cross_entropy(logits, targets) # torch wants B x T X C. 
            # For each row of logits, Treat it as the unnormalized scores for all 65 tokens. Apply softmax internally → get probabilities over 65 classes
            # for embeddings of other size 128 dimension etc, we have a linear neural network that projects it back to logits
            # self.fc = nn.Linear(embedding_dim, vocab_size)  # 124 → 65
            # logits = self.fc(embedding_table[idx])          # shape: (B, T, 65)
            # omg that Problem: You cannot feed 124-dimensional embeddings directly to cross-entropy, because cross_entropy expects logits over vocab size (65).
            # Solution: add a linear projection: totally solved all my confusion. karpathy basically left this critical step, or he had another video which i 
            # didnt watch. so basically the embedding is like a input for neural network with its own layers . can be multiple and that outputs the logits and we do 
            # softmax to get max probability of token and compare that to target. 
            # solved. all clear now only tell me in this case the input is independently used to generate output token by token 
            # 24, 43, 58,  5, 57,  1, 46, 43 - are we brute forcing it. like for ex  we use embedding of 58 only to generate the output and not 24,43,58
            # The model is very simple: it only looks at the current token to predict the next token
            # Input 24 → embedding → predicts next token 43
            # Input 43 → embedding → predicts next token 58
            # Input 58 → embedding → predicts next token 5
            # … and so on
            # ✅ So yes, each step is independent in terms of what embedding is used: only the current token’s embedding is fed into the model to predict the next token.
                        
        return logits, loss 

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the latest block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx) # goes to forward function
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C) # pick out only the last element of a B x T  x C logit a  (B, C) = (4, 65) tensor if input is 4 x 8 x 65
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C) # logits to softmax. dim=-1 → apply softmax along the last dimension (here, C, vocab dimension)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) # get only one
            # torch.multinomial samples from a categorical distribution defined by probs
            # we want the model’s prediction to guide the choice: tokens with higher probabilities are more likely to be picked.
            # probs = [0.7, 0.1, 0.1, 0.1]  # vocab size = 4
            # torch.multinomial(probs, 1)   # 70% chance to pick token 0, 10% chance for 1,2,3
            # If you always picked argmax, it would be deterministic. Multinomial allows stochastic sampling, which is important for generating diverse tex
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1) concatenate along seq dimension dim=1 → new idx is (B, T+1) = (4, 9)
        return idx

m = BigramLanguageModel(vocab_size)
m = m.to(device)
logits, loss = m(xb, yb) # a forward pass with 4 x 8 matrix
# tensor([[24, 43, 58,  5, 57,  1, 46, 43],
#         [44, 53, 56,  1, 58, 46, 39, 58],
#         [52, 58,  1, 58, 46, 39, 58,  1],
#         [25, 17, 27, 10,  0, 21,  1, 54]])
# targets:
# torch.Size([4, 8])
# tensor([[43, 58,  5, 57,  1, 46, 43, 39],
#         [53, 56,  1, 58, 46, 39, 58,  1],
#         [58,  1, 58, 46, 39, 58,  1, 46],
#         [17, 27, 10,  0, 21,  1, 54, 39]])
print(loss) # we get scores for torch.Size([4, 8, 65])
idx = torch.zeros((1,1), dtype=torch.long) # 0 is a newline character and we start 
idx = torch.tensor([[5]]) # 5 as the starting
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))
# this is without history
# bigram model. this looks only at last character to predict next. we expand to use history to predict next token


TypeError: BigramLanguageModel.__init__() takes 1 positional argument but 2 were given

In [ ]:
# Optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # for smaller network higher LR 


In [ ]:
# # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
# print(device)

In [ ]:

import time

print("Using device:", device)


batch_size = 128  # bigger batch helps GPU utilization
steps = 10000



start_time = time.time()
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets. interval is set to 300, so 10 times loss between 0 to 3000 iterations
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch_device('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True) # zeroing up the gradients in previous step
    loss.backward()
    optimizer.step()


end_time = time.time()
print(f"Training finished in {end_time - start_time:.2f} seconds")
# 2 min 34 secs for cuda 4.6973 to 2.4142. 2 min 20.3 secs in CPU 4.6973 to 2.4142


Using device: cpu
Step 0, Loss: 4.6973


Step 100, Loss: 4.4980
Step 200, Loss: 4.3540
Step 300, Loss: 4.3019
Step 400, Loss: 4.1383
Step 500, Loss: 4.0842
Step 600, Loss: 3.9211
Step 700, Loss: 3.8928
Step 800, Loss: 3.7173
Step 900, Loss: 3.6722
Step 1000, Loss: 3.5917
Step 1100, Loss: 3.4900
Step 1200, Loss: 3.4401
Step 1300, Loss: 3.3894
Step 1400, Loss: 3.3141
Step 1500, Loss: 3.2329
Step 1600, Loss: 3.1788
Step 1700, Loss: 3.1531
Step 1800, Loss: 3.0930
Step 1900, Loss: 3.0790
Step 2000, Loss: 2.9576
Step 2100, Loss: 2.9700
Step 2200, Loss: 2.8952
Step 2300, Loss: 2.8947
Step 2400, Loss: 2.8516
Step 2500, Loss: 2.8409
Step 2600, Loss: 2.8163
Step 2700, Loss: 2.6917
Step 2800, Loss: 2.7242
Step 2900, Loss: 2.6964
Step 3000, Loss: 2.6581
Step 3100, Loss: 2.6654
Step 3200, Loss: 2.6539
Step 3300, Loss: 2.6368
Step 3400, Loss: 2.6182
Step 3500, Loss: 2.5721
Step 3600, Loss: 2.5843
Step 3700, Loss: 2.5734
Step 3800, Loss: 2.5613
Step 3900, Loss: 2.5380
Step 4000, Loss: 2.6231
Step 4100, Loss: 2.5287
Step 4200, Loss: 2.5319
S

In [ ]:
# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

#initially the output is 
# 'XAQHukRuaRJKXAYtXzfJ:HEPiu--sDioi;ILCo3pHNTmDwJsfheKRxZCFs
# lZJ XQc?:s:HEzEnXalEPklcPU cL'DpdLCafBheH

# after 10000 steps it is 
# I yome k ee mprop ves.
# med belenk s w me ave hirmin, baveard.
# bellfushealmiere ENus
# BAy he,
# NARWhan 


I yome k ee mprop ves.
med belenk s w me ave hirmin, baveard.
bellfushealmiere ENus
BAy he,
NARWhan 


In [ ]:
# batch_size = 32
# # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# m = m.to(device)
# for steps in range(10000):
#     # sample a batch
#     xb, yb = get_batch('train')
#     xb = xb.to(device)
#     yb = yb.to(device)
#     # training loop
#     logits, loss = m(xb, yb)
#     optimizer.zero_grad(set_to_none=True) # zeroing up the gradients in previous step
#     loss.backward()
#     optimizer.step() # update parameters
#     print(loss.item())
# # loss goes from for 3.32 to 3.27 in 100 iterations
# # 10,000 steps need 1 min 7.5 seconds in CPU. with cuda it is 1 min 11.2 secs for 10,000 from start. loss went from 4.67 to 2.54

In [ ]:
# The tokens are not taking to each other. now they talk to each other and figure out what is in context and predict better
# estimate loss in multiple batches - trainand validation loss. torch.no_grad() in estimate loss
# token in 5th location should talk only to 1,2,3,4 and not 6,7,8
# 5th token - 1,2,3,4 averaged out info like a feature. every t tokens calculate avg of vectors

B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B): # for each  batch
    for t in range(T): # for each block from first to cumulative of last 
        xprev = x[b,:t+1] # (t,C) always a t+1 x c where t ranges from 0 to some number 
        xbow[b,t] = torch.mean(xprev, 0) # torch.mean(xprev, 0) computes the mean along the first dimension (rows). It sums up each column of xprev (element-wise across the 4 rows).
        #Then divides by the number of rows (t+1 = 4) → gives average per column.

In [ ]:
print(x[0])
print(xbow[0])
# tensor([[ 0.1808, -0.0700],
#         [-0.3596, -0.9152],
#         [ 0.6258,  0.0255],
#         [ 0.9545,  0.0643],
#         [ 0.3612,  1.1679],
#         [-1.3499, -0.5102],
#         [ 0.2360, -0.2398],
#         [-0.9211,  1.5433]])
# tensor([[ 0.1808, -0.0700],
#         [-0.0894, -0.4926],
#         [ 0.1490, -0.3199],
#         [ 0.3504, -0.2238],
#         [ 0.3525,  0.0545],
#         [ 0.0688, -0.0396],
#         [ 0.0927, -0.0682],
#         [-0.0341,  0.1332]])

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])
tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])


In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
# torch.ones give 3 x 3 of 1 s
# troch.tril gives a lower quadrant 1s and upper zeros
a = a / torch.sum(a, 1, keepdim=True)
# this gives the avg of each row in place of 1s, first row has one 1 so it is 1, second row has two 1s so 0.5 , 0.5 each row adds up to 1. so in this way our matrix 
# multiplication gives avg till that number , basically what we did using for loop above
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T)) # using the above concept we get T x T - each row
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C). Torch auto adds B times T x T to wei 
torch.allclose(xbow, xbow2)

In [ ]:
# version 3: use Softmax
# weighted aggregation of lower triangle. the -inf mask is to ensure future tokens are not communicating with past when we do weighted average
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
# wei =
# [[1.0000, 0.0000, 0.0000],
#  [0.5000, 0.5000, 0.0000],
# #  [0.3333, 0.3333, 0.3333]]
# the end result is averaged of previous context till current token

xbow3 = wei @ x
torch.allclose(xbow, xbow3)

In [ ]:
# Self attention
# version 4: different importance instead of uniform averaging out
# Self attentin. every single token emits a query, key
# query - what am i looking for
# key - what do i contain
# dot product query with all keys . this becomes wei. if key and query are aligned then we get higher

B, T, C = 4, 8, 32
x = torch.randn(B,T,C)

# Single Self Attention Head
head_size = 16
key = nn.Linear(C, head_size, bias=False) # These take in a C and output head_size which is 16
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # B x T x 16
q = query(x) # B x T x 16
wei = q @ k.transpose(-2,-1) * head_size**-0.5#. k.transpose(−2,−1): swap the last two dims → (B,16,T) 
# Torch batch matrix multiplication for each Batch we have a T x 16 and we multiply it by a 16 x T which results in a T x T 
# B , T, 16 x B x 16 X T --> B , T , T 
# the division by sqrt head_size is done as if not the softmax sharpens or gets a higher probability for the highest value much higher than others
# so the aggregation of information is focussed on only on the biggest value instead of spread out
tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
v = value(x)
# out = wei @ x # we dont aggregate tokens
out = wei @ v # we agregate values according to wei
print(out) # 4 x 8 x 16 dim . heres what i am interested in, heres what i have, heres what i will communicate if you find me interesting (V). V is getting aggregated here


tensor([[[ 4.3612e-01,  2.6055e-01, -8.7791e-02,  2.4081e-01, -1.2670e-01,
          -6.3213e-01,  2.5118e-01, -3.7007e-01,  1.5550e-01,  7.7529e-01,
           3.9655e-01,  4.1202e-01, -9.7145e-02,  2.7214e-01, -9.1076e-01,
          -5.7588e-01],
         [ 4.0732e-01,  1.7379e-01,  1.4678e-01,  1.9324e-01, -1.1125e-01,
          -2.2321e-01,  3.1712e-01, -4.8082e-01,  3.4286e-01,  7.2147e-01,
           1.8975e-01,  2.7924e-01, -6.0415e-02,  3.9078e-01, -6.3641e-01,
          -3.1325e-01],
         [ 1.1724e-01,  7.4618e-02,  3.1544e-01,  3.7297e-01, -8.5293e-02,
           2.0070e-01,  7.6096e-02, -7.3248e-01,  1.8252e-01,  6.1751e-01,
          -3.5668e-01, -1.9321e-01,  1.6650e-01,  5.6478e-01, -2.0256e-01,
           8.7309e-03],
         [-6.7887e-04,  5.5931e-02,  2.1709e-01,  3.9189e-01,  8.5315e-02,
           2.3985e-01,  1.5639e-02, -6.5458e-01,  2.7192e-02,  6.7284e-01,
          -3.4941e-01, -1.6289e-01,  2.4242e-01,  5.4952e-01, -1.9463e-02,
           2.3055e-01],
    

In [27]:
print(v)

tensor([[[ 4.3612e-01,  2.6055e-01, -8.7791e-02,  2.4081e-01, -1.2670e-01,
          -6.3213e-01,  2.5118e-01, -3.7007e-01,  1.5550e-01,  7.7529e-01,
           3.9655e-01,  4.1202e-01, -9.7145e-02,  2.7214e-01, -9.1076e-01,
          -5.7588e-01],
         [ 3.0401e-01, -1.3744e-01,  9.8834e-01,  2.2601e-02, -5.5825e-02,
           1.2438e+00,  5.5367e-01, -8.7818e-01,  1.0150e+00,  5.2840e-01,
          -5.5217e-01, -1.9714e-01,  7.1362e-02,  8.1641e-01,  3.4788e-01,
           6.2898e-01],
         [-5.0317e-01,  1.6584e-02,  2.3979e-01,  8.8837e-01, -5.5558e-02,
           3.7241e-01, -6.1939e-01, -1.0989e+00, -5.6760e-01,  4.8231e-01,
          -1.2197e+00, -1.0316e+00,  6.2335e-01,  7.3385e-01,  2.6198e-01,
           2.3525e-01],
         [-6.1592e-01,  5.6083e-02, -5.8237e-01,  5.6702e-01,  9.7417e-01,
          -2.4427e-02, -4.0814e-01, -1.2484e-01, -1.0444e+00,  1.0246e+00,
          -1.0066e-01,  1.2252e-01,  6.1935e-01,  3.3588e-01,  6.4877e-01,
           1.1093e+00],
    

In [ ]:
print(wei[0])
# attention is a communication mechanism . every node as vector of info, we aggregate info as weighted sum from all nodes pointing to it
# attention acts over set of vectors without notion of space, so we positionally encode tokens
# element across batch dont talk to each other. we have 4 pool of 8 nodes, who only talk among those 8 within the batch. only to past
# sentiment analysis - all token talk to each other. in encoder block we delete wei = wei.masked_fill(tril == 0, float('-inf')) to enable 
# cross attention - q,k,v from same source x. self attended. in encoded-decoder block. q come from some source and k,v from another source. seperate source
# where we pool info
# now this contains diff values - information aggregated from tokens in past
# tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
#         [0.4822, 0.5178, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
#         [0.5354, 0.2838, 0.1808, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
#         [0.1647, 0.0943, 0.1177, 0.6233, 0.0000, 0.0000, 0.0000, 0.0000],
#         [0.7846, 0.0586, 0.0478, 0.0476, 0.0614, 0.0000, 0.0000, 0.0000],
#         [0.1013, 0.2149, 0.1094, 0.3307, 0.1485, 0.0952, 0.0000, 0.0000],
#         [0.0458, 0.0425, 0.2363, 0.1927, 0.1673, 0.1253, 0.1901, 0.0000],
#         [0.0423, 0.0665, 0.1825, 0.0205, 0.1035, 0.0896, 0.2237, 0.2714]],

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4822, 0.5178, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5354, 0.2838, 0.1808, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1647, 0.0943, 0.1177, 0.6233, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7846, 0.0586, 0.0478, 0.0476, 0.0614, 0.0000, 0.0000, 0.0000],
        [0.1013, 0.2149, 0.1094, 0.3307, 0.1485, 0.0952, 0.0000, 0.0000],
        [0.0458, 0.0425, 0.2363, 0.1927, 0.1673, 0.1253, 0.1901, 0.0000],
        [0.0423, 0.0665, 0.1825, 0.0205, 0.1035, 0.0896, 0.2237, 0.2714]],
       grad_fn=<SelectBackward0>)
